In [4]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
import json
from Create_Table_queries import *

%load_ext autoreload
%autoreload 2

def process_youtubedata_file(cur, filepath): 
    """
        This function reads one JSON file and read information of videos and youtuber data and saves into video_data and youtuber_data
        Arguments:
        cur: Database Cursor
        filepath: location of JSON files
        Return: None
    """
    # open JSON file
    df = pd.read_json(filepath, lines=True)


    # ---------insert youtuber record----------
    # write your code here that reads youtuber data from JSON file and insert it into Youtubers_dim table 
    # write your code here
    for key, item in df.iterrows():
        youtuber_id=item['youtuber_id']['0'];
        location=item['youtuber_location']['0'];
        latitude=item['youtuber_latitude']['0'];
        longitude=item['youtuber_longitude']['0'];
        name=item['youtuber_name']['0'];

        youtuber_record=(youtuber_id,name,location,latitude,longitude)
        cur.execute(Youtubers_table_insert, youtuber_record)

        # ---------insert video record--------------
        # write your code here that reads youtube videos data from JSON file and insert it into Videos_dim table 
        # write your code here
        video_id=item['video_id']['0'];
        title=item['title']['0'];
        year=item['year']['0'];
        duration=item['duration']['0'];

        video_record=(video_id,title,youtuber_id,year,duration)
        cur.execute(Videos_table_insert, video_record)


def process_log_file(cur, filepath):
    """
        This function reads Log files and reads information of time, user and videoplay data and saves into time, user, videoplay
        Arguments:
        cur: Database Cursor
        filepath: location of Log files
        Return: None
    """

    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextVideo action
    df = df[(df['page'] == 'NextVideo')]
    
    

    # convert timestamp column to datetime with 'pd.to_datetime(df["ts"], unit="ms")'   
    # insert time data records to Time_dim table
    
    
    # write your code here
    for key, item in df.iterrows():
        start_time=item["ts"]
        hour=pd.to_datetime(item["ts"], unit="ms").hour
        day=pd.to_datetime(item["ts"], unit="ms").day
        week=pd.to_datetime(item["ts"], unit="ms").week
        month=pd.to_datetime(item["ts"], unit="ms").month
        year=pd.to_datetime(item["ts"], unit="ms").year
        weekday=pd.to_datetime(item["ts"], unit="ms").dayofweek
        time_record=(start_time,hour,day,week,month,year,weekday)
        #print(Time_table_insert,time_record)
        cur.execute(Time_table_insert, time_record)
      
    # load user table
    # insert user records into Users_dim table

    
        # write your code here
        user_id=item["userId"]
        first_name=item["firstName"]
        last_name=item["lastName"]
        gender=item["gender"]
        level=item["level"]
        user_record=(user_id,first_name,last_name,gender,level)
        #print(Time_table_insert,time_record)
        cur.execute(Users_table_insert, user_record)

    
    # insert Videoplay records in Videoplay_fact table

    
        # write your code here
        video_id=""
        youtubers_id=""
        session_id=item["sessionId"]
        location=item["location"]
        user_agent=item["userAgent"]
        
        cur.execute("SELECT video_id FROM Videos_dim where title like %s Limit 1",[item["video"]])
        rows = cur.fetchall()
        for row in rows:
               video_id=row[0]
        cur.execute("SELECT youtuber_id FROM Youtubers_dim where name like %s Limit 1",[item["youtuber"]])
        rows = cur.fetchall()
        for row in rows:
               youtubers_id=row[0]       
        video_play_record=(start_time,user_id,level,video_id,youtubers_id,session_id,location,user_agent)
        #print(Videoplay_table_insert,video_play_record)
        cur.execute(Videoplay_table_insert, video_play_record)


def process_data(cur, conn, filepath, func):
    """
        This function get all JSON files in given directory by exploring all sub directories, and process all files that were found using the given function.
        Example: if I give it the path to youtube_data directory which resides in data folder of this assignment,
        and func given is process_youtubedata_file it should get all JSON files in this directories and process each file using process_youtubedata_file function. 
        Arguments:
        cur: Database Cursor
        conn: Database
        filepath: location of JSON files
        func: function to process all files in the directory
        Return: None
    """
    for path, subdirs, files in os.walk(filepath):
        for name in files:
            func(cur,os.path.join(path, name))
    conn.commit()

def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=12345678")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/youtube_data', func=process_youtubedata_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
